<span style="font-size:large;">
<div style="text-align: right">IM-UH 1511 <b>Introduction to Digital Humanities</b><br/>
Student name: <span style="color:blue"><b>Ben Pang</b></span><br/>
Grade: <span style="color:red"><b></b></span><br/><br/></div>

## <span style="color:red">HOMEWORK 1</span>
# <span style="color:green">Extraction of Names from Text

In [1]:
import time
start_time = time.time()
import urllib, os, codecs, random, operator, re, string, copy, dateutil.parser, itertools, pickle, datetime, math, pandas as pd, numpy as np, matplotlib.pyplot as plt
from collections import Counter
from string import punctuation, digits
import pathlib
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')
import inflect
import nltk
from nltk import word_tokenize

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.simplefilter('ignore')

### Load Data

In [2]:
# get your working directory
home = str(pathlib.Path.cwd())

# create a path to which the file will be written
text_path = os.path.join(home, 'pride_and_prejudice.txt')

# location of the project gutenberg copy of the pride & prejudice text file
text_url = 'http://www.gutenberg.org/files/1342/1342-0.txt'

urllib.request.urlretrieve(text_url, text_path)

print('Downloaded to:', text_path)

RemoteDisconnected: Remote end closed connection without response

In [ ]:
f = codecs.open(text_path, "r", encoding="utf-8").readlines()
start, end = 0, 0
for line in f:
    if line.startswith("      Chapter 1"):
        start = f.index(line)
        print(start)
    if line.startswith("End of the Project"):
        end = f.index(line)
        print(end)

In [ ]:
ff=f[start:end]
ff

In [ ]:
ff[-10:]

In [ ]:
text="\r\n".join(ff)
text

In [ ]:
titlename = "Jane Austen's Pride and Prejudice"

words = word_tokenize(text)
nuw=len(words)
uw=len(set(words))
print("%s contains %i nonunique and %i unique words"%(titlename,nuw,uw))

### Extraction of Proper Nouns

In [ ]:
p = inflect.engine()
d_tags = {}

docs_d={"Pride and Prejudice":text}
for key, value in docs_d.items():
    arr = []
    doc = nlp(value.replace('\n',''))
    #Keep these types of nlp entities
    keep_l = ['PERSON'] #,'NORP','PRODUCT','ORG']
    #Typo/model error + german corrections
    drop_t = []
    
    #Things inflect library handles poorly or to exclude from touching
    ex_ls = []
    
    for X in doc.ents:
        s1 = X.text
        if (X.label_ in keep_l) and (s1.lower() not in drop_t) and (s1):
            arr.append((s1, X.label_))
    d_tags[key] = arr
# pprint(d_tags)
names=[]
for k,v in d_tags.items():
    for vv in v:
        if vv[0] not in names:
            p=vv[0].replace("'","")
            p=p.title()
            names.append(p)
names=sorted(set(names))
print(len(names))
names

In [ ]:
rem=[]
for p in names:
    if "_" in p:
        rem.append(p)
    if "--" in p:
        rem.append(p)
    if p not in text:
        rem.append(p)
names=[p for p in names if p not in rem]
pp=[q for q in itertools.product(names,names) if q[0]!=q[1]]
for q in pp:
    if q[0] in q[1]:
        rem.append(q[0])
    if q[1] in q[0]:
        rem.append(q[1])
    w=q[0]+" "+q[1]
    if w in text:
        names.append(w)
        rem.append(q[0])
        rem.append(q[1])
names=[p for p in names if p not in rem]
names=sorted(set(names))
print(len(names))
names

In [ ]:
rem=['Esq', 'F.', 'Follies', 'Nay', 'Niece', 'Till Elizabeth', 'W.']
names=[p for p in names if p not in rem]
names=names+[]
names=sorted(set(names))
print(len(names))
names

In [ ]:
nfreq=[]
for i in names: 
    nfreq.append(text.count(i))
pnf_df = pd.DataFrame(
    {'Names': names, 
     'Frequency of Occurrences': nfreq
    })
pnf_df=pnf_df[['Names','Frequency of Occurrences']]
pnf_df=pnf_df.sort_values(by ='Frequency of Occurrences',ascending=False)
# trf_df=trf_df[trf_df["Frequency of Occurrences"]>10]
print(len(pnf_df))
pnf_df[:50]

In [ ]:
pnf_df.to_csv('Names_freqs.csv')

In [ ]:
print("Run in %.2f seconds (%.2f minutes)" %(time.time() - start_time,(time.time() - start_time)/60))